# **Running Jupyter Code with Ollama Server**

1. **Download and Install Ollama**: Ensure that Ollama is downloaded and installed on your system.

2. **Check Ollama Version**:
   ```bash
   ollama --version
   ```

3. **Run Llama 3.2 Model**:
   To load the Llama 3.2 model:
   ```bash
   ollama run llama3.2
   ```

4. **Start Ollama Server**:
   To start the Ollama server, run:
   ```bash
   ollama serve
   ```

**Note**: If you face any issues while running the server, checking the background tasks using the port can help resolve conflicts. Always ensure the port is free before starting the server.

1. **Check if Ollama Server is Running**:
   Check if the server is running and using port 11434:
   ```bash
   netstat -ano | findstr :11434
   ```

2. **Identify and Kill Background Task (if needed)**:
   If you see that another task is using the port, you can identify and kill it:
   ```bash
   tasklist /FI "PID eq <PID>"
   taskkill /PID <PID> /F
   ```

3. **Close Ollama from Windows Task Bar (Optional)**:
   If the server is still not responding or you see issues, you can use the **up arrow key** (^) on the Windows task bar (bottom-left) to close any running Ollama instances.

4. **Start Jupyter Notebook**:
   Once the server is running, start your Jupyter notebook with the necessary code to interact with Ollama.

In [1]:
import ollama
from PyPDF2 import PdfReader
from IPython.display import display, Markdown ,update_display

In [2]:
system_prompt = '''You are an agent which analyses the resume and understands the context of resume and 
    extract the following fields and format the output:
    - Name
    - Email
    - Phone 1
    - Phone 2
    - Address
    - City
    - LinkedIn
    - Professional Experience (in years)
    - Highest Education
    - Is Fresher (yes/no)
    - Is Student (yes/no)
    - Skills (comma-separated)
    - Applied For Profile
    - Education (Institute Name, Year of Passing, Score)
    - Projects (Project Name, Description)
    - Professional Experience (Organisation Name, Duration, Profile)

    Ensure to format the output as:
    Name: [Value]
    Email: [Value]
    Phone 1: [Value]
    ... and provide this summary of resume in markdown format'''

In [3]:
def extract_text_from_pdf(pdf_path):
    text = ""
    try:
        with open(pdf_path, "rb") as file:
            reader = PdfReader(file)
            for page in reader.pages:
                text += page.extract_text()  # Extract text from each page
    except Exception as e:
        print(f"Error extracting text from PDF: {e}")
    return text
    
user_prompt = extract_text_from_pdf('./data/python-developer-resume-example.pdf') + "Provide a summary of this resume"

In [ ]:
import requests
import json

def call_ollama(system_prompt, user_prompt):
    url = "http://localhost:11434/api/chat"
    
    payload = {
        "model": "llama3.2",
        "messages": [
            {
                "role": "system",
                "content": system_prompt
            },
            {
                "role": "user",
                "content": user_prompt
            }
        ]
    }
    
    response = requests.post(url, json=payload)
    
    if response.status_code == 200:
        full_response = ""
        for line in response.iter_lines():
            if line:
                json_response = line.decode('utf-8')
                try:
                    response_data = json.loads(json_response)
                    if 'message' in response_data:
                        full_response += response_data['message'].get('content', '')
                except json.JSONDecodeError:
                    print("Error decoding JSON")
        
        return display(Markdown(full_response))
    else:
        print(f"Error: {response.status_code}")
        return None


response = call_ollama(system_prompt, user_prompt)
response

## **OLD CODE**

**NOTE :** Just Kept the old code to show how Ollama provdes answers it sends one word at a time in a chunk type response 

In [ ]:
import requests

url = "http://localhost:11434/api/chat"
headers = {
    "Content-Type": "application/json"
}

# Define the conversation prompt and specify the model
data = {
    "model": "llama3.2",  # Specify the model name here
    "messages": [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "What is the capital of France?"}
    ]
}

# Send the POST request to the Ollama server
response = requests.post(url, json=data, headers=headers)

# Print the raw response text for debugging
print("Raw Response:", response.text)

# Try to parse the response only if it's in JSON format
try:
    response_json = response.json()  # Try parsing the response as JSON
    print("Model Response:", response_json)
except ValueError as e:
    print("Error parsing JSON:", e)
    print("Response text may not be valid JSON.")


## **Easier Way**

In [5]:
import ollama

def call_ollama(system_prompt, user_prompt):
    stream = ollama.chat(model='llama3.2', messages=[
        {
            'role': 'system',
            'content': system_prompt
        },
        {
            'role': 'user',
            'content': user_prompt
        }
    ])

    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream['message']['content']:
        response += chunk or''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)
        
# # Example usage
system_prompt = "You are a helpful assistant who explains complex topics in simple terms."
user_prompt = "Who is Led Zepplin"

response = call_ollama(system_prompt, user_prompt)
response

Led Zeppelin, often shortened to Led Zeppelin or simply Zep, was a legendary British rock band formed in London in 1968. The band is widely regarded as one of the most influential and iconic rock bands in history.

Here's a brief overview:

**The Band:**

* Robert Plant (vocals)
* Jimmy Page (guitar, production)
* John Paul Jones (bass guitar, keyboards, production)
* John Bonham (drums)

**Music Style:**
Led Zeppelin was known for their unique blend of blues, folk, and hard rock music. They were heavily influenced by American blues musicians like Robert Johnson and Muddy Waters, as well as classical composers like Richard Wagner.

**Notable Songs:**

* "Stairway to Heaven"
* "Whole Lotta Love"
* "Kashmir"
* "Black Dog"
* "Rock and Roll"

**Impact:**
Led Zeppelin's music had a profound impact on the development of rock music. They helped shape the genre, pushing boundaries with their innovative use of guitar, bass, and drums. Their powerful live performances and iconic music videos (for their time) helped establish them as one of the most influential bands in history.

**Legacy:**
Led Zeppelin's legacy extends far beyond their music. The band has inspired countless other artists, including rock, metal, and hard rock musicians. They've also influenced numerous films, TV shows, and even fashion trends (e.g., Robert Plant's iconic outfits).

Despite the tragic passing of John Bonham in 1980 and Jimmy Page's departure from the band, Led Zeppelin continues to be celebrated by fans worldwide.

Are you a fan of Led Zeppelin? Do you have a favorite song or album?